In [1]:
from __future__ import print_function

"""
A reference implementation of cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_cis import *
from helper_cs_cqed_cis import *
from psi4.driver.procrouting.response.scf_response import tdscf_excitations
import fortecubeview
# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
#psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


In [3]:

mol_str = """

0 1
O 0.0000000000 0.0000000000 0.6881243879
C 0.0000000000 0.0000000000 -0.5150066070
H 0.0000000000 0.9462463671 -1.1029423155
H 0.0000000000 -0.9462463671 -1.1029423155
no_reorient
symmetry c1
"""
# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk'}

# set psi4 options and geometry
psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)
psi4.optimize('scf/cc-pvdz', molecule=mol)

a = np.asarray(mol.geometry())
print(a * 0.529)

Optimizer: Optimization complete!
[[ 0.00000000e+00 -1.91482801e-12  5.91268220e-01]
 [ 0.00000000e+00  2.36426838e-12 -5.90400099e-01]
 [ 0.00000000e+00  9.32184336e-01 -1.17703144e+00]
 [ 0.00000000e+00 -9.32184336e-01 -1.17703144e+00]]


In [8]:
mol_str = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# set psi4 options and geometry
psi4.set_options(options_dict)
mol = psi4.geometry(mol_str)
# photon energy
om = 0 #4.34405539 / 27.211

# electric field strength 
#E_au = 0.00

# strength for the dipole self energy
#l = E_au / np.sqrt(om/2)
l = 0 #.01
lam = np.array([0,0,l])

# run psi4 SCF
psi4_rhf_e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
# confirm our cis agrees with psi4
res = tdscf_excitations(wfn, states=3, triplets = "NONE", tda=True)
    
# parse res for excitation energies
psi4_excitation_e = [r["EXCITATION ENERGY"] for r in res]
    
# run helper_cs_cqed_cis()
#cqed_dict = cs_cqed_cis(lam, mol_str, options_dict, om)

#cqed_cis_e = cqed_dict['cqed_cis excitation energies']
#scf_e = cqed_dict['rhf_energy']

print(res)

[{'EXCITATION ENERGY': 0.173707477525952, 'ELECTRIC DIPOLE TRANSITION MOMENT (LEN)': array([-2.79174283e-16, -2.37642936e-16, -2.66241308e-17]), 'OSCILLATOR STRENGTH (LEN)': 1.5647718897011975e-32, 'ELECTRIC DIPOLE TRANSITION MOMENT (VEL)': array([ 9.23386636e-17, -6.93258461e-17,  5.75986118e-17]), 'OSCILLATOR STRENGTH (VEL)': 6.39009214789298e-32, 'MAGNETIC DIPOLE TRANSITION MOMENT': array([2.02228246e-16, 4.88154671e-15, 5.91396406e-01]), 'ROTATORY STRENGTH (LEN)': -1.5745415243787747e-17, 'ROTATORY STRENGTH (VEL)': -1.9609755720401918e-16, 'SYMMETRY': 'A', 'SPIN': 'singlet', 'RIGHT EIGENVECTOR ALPHA': <psi4.core.Matrix object at 0x7ffbb85c5ae0>, 'LEFT EIGENVECTOR ALPHA': <psi4.core.Matrix object at 0x7ffbb85c5ae0>, 'RIGHT EIGENVECTOR BETA': <psi4.core.Matrix object at 0x7ffbb85c5ae0>, 'LEFT EIGENVECTOR BETA': <psi4.core.Matrix object at 0x7ffbb85c5ae0>}, {'EXCITATION ENERGY': 0.37801837301368574, 'ELECTRIC DIPOLE TRANSITION MOMENT (LEN)': array([ 4.58108682e-02, -4.58814223e-14, -7

In [9]:
print(cqed_cis_e)

[ 0.          0.          0.17370801  0.17370801  0.37801968  0.37801968
  0.38038117  0.38038117  0.38795265  0.38795265  0.43760533  0.43760533
  0.47316602  0.47316602  0.47358652  0.47358652  0.49106495  0.49106495
  0.54424724  0.54424724  0.58190264  0.58190264  0.61521332  0.61521332
  0.61729153  0.61729153  0.65916339  0.65916339  0.67865941  0.67865941
  0.69094373  0.69094373  0.71703615  0.71703615  0.74134696  0.74134696
  0.75824765  0.75824765  0.8017917   0.8017917   0.83479393  0.83479393
  0.85385542  0.85385542  0.86902137  0.86902137  0.89560552  0.89560552
  0.92183347  0.92183347  0.92742369  0.92742369  0.93355667  0.93355667
  0.93783667  0.93783667  0.96098016  0.96098016  0.96191048  0.96191048
  0.97851139  0.97851139  1.01691237  1.01691237  1.01715887  1.01715887
  1.02688294  1.02688294  1.04929776  1.04929776  1.05200378  1.05200378
  1.07150505  1.07150505  1.11156855  1.11156855  1.1180666   1.1180666
  1.13217013  1.13217013  1.14678205  1.14678205  1.

In [23]:
mol_str = """

0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1
"""

# photon energy resonant with first CIS transition energy
om = 0.0
print(om)

# electric field strength 
E_au = 0.01

# strength for the dipole self energy
#l = E_au / np.sqrt(om/2)

lam = np.array([0,0,0])

# run psi4 SCF
psi4_rhf_e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
    
# run helper_cs_cqed_cis()
cqed_dict = cs_cqed_cis(lam, mol_str, options_dict, om)

cqed_cis_e = cqed_dict['cqed_cis excitation energies']
scf_e = cqed_dict['rhf_energy']

0.0
-113.8772221762759

Start SCF iterations:

Canonical RHF One-electron energy = -218.2835951352540746
CQED-RHF One-electron energy = -218.2835951352540746
Nuclear repulsion energy = 31.7803657982661569
Dipole energy = 0.0000000000000000
SCF Iteration   1: Energy = -113.8772221766076882   dE = -1.13877E+02   dRMS = 8.64881E-08
SCF Iteration   2: Energy = -113.8772221766161010   dE = -8.41283E-12   dRMS = 3.57082E-08
Performed QED-RHF on the following molecule


0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1

Total time for SCF iterations: 0.022 seconds 

QED-RHF   energy: -113.87722218 hartree
Psi4  SCF energy: -113.87722218 hartree


In [27]:
print(cqed_cis_e[0:8]+0.38/27.211+-113.87722218)

[-113.86325724 -113.86325724 -113.68954923 -113.68954923 -113.48523756
 -113.48523756 -113.48287607 -113.48287607]


In [30]:
Ex = np.linspace(0, 0.02, 20) #np.array([0, 0.002, 0.004, 0.008, 0.012])
f = open('formaldehyde_ccpVDZ_Ey.txt', 'w')
om = 0.38/27.211
for vx in Ex:
    lx = vx / np.sqrt(om/2)
    #ly = vy / np.sqrt(om/2)
    lam = np.array([0,ly,0])
    cqed_dict = cs_cqed_cis(lam, mol_str, options_dict, 0.38/27.211)
    cqed_cis_e = cqed_dict['cqed_cis excitation energies']
    cqed_scf_e = cqed_dict['cqed_rhf_energy']
    f.write("%6.3f %6.3f %12.6f %12.6f %12.6f %12.6f %12.6f %12.6f %12.6f %12.6f\n" % (vx,lx,cqed_cis_e[0]+cqed_scf_e,cqed_cis_e[1]+cqed_scf_e,
            cqed_cis_e[2]+cqed_scf_e,cqed_cis_e[3]+cqed_scf_e,
            cqed_cis_e[4]+cqed_scf_e,cqed_cis_e[5]+cqed_scf_e,
            cqed_cis_e[6]+cqed_scf_e,cqed_cis_e[7]+cqed_scf_e))
f.close()
        
        
# run helper_cs_cqed_cis()
#cqed_dict = cs_cqed_cis(lam, mol_str, options_dict, om)

#cqed_cis_e = cqed_dict['cqed_cis excitation energies']
#scf_e = cqed_dict['rhf_energy']

-113.8772221762759

Start SCF iterations:

Canonical RHF One-electron energy = -218.2835951352540746
CQED-RHF One-electron energy = -218.2780272733479308
Nuclear repulsion energy = 31.7803657982661569
Dipole energy = 0.0000000000000000
SCF Iteration   1: Energy = -113.8745521325829202   dE = -1.13875E+02   dRMS = 3.86627E-05
SCF Iteration   2: Energy = -113.8745544215829284   dE = -2.28900E-06   dRMS = 1.45431E-05
SCF Iteration   3: Energy = -113.8745546226130330   dE = -2.01030E-07   dRMS = 6.74540E-06
SCF Iteration   4: Energy = -113.8745546510455426   dE = -2.84325E-08   dRMS = 4.69966E-06
Performed QED-RHF on the following molecule


0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1

Total time for SCF iterations: 0.045 seconds 

QED-RHF   energy: -113.87455465 hartree
Psi4  SCF energy: -113.87722218 hartree
-113.8772221762759

-113.8772221762759

Start SCF iterations:

Canonical RHF One-electron energy = -218.2835951352540746
CQED-RHF One-electron energy = -218.2780272733479308
Nuclear repulsion energy = 31.7803657982661569
Dipole energy = 0.0000000000000000
SCF Iteration   1: Energy = -113.8745521325829202   dE = -1.13875E+02   dRMS = 3.86627E-05
SCF Iteration   2: Energy = -113.8745544215829284   dE = -2.28900E-06   dRMS = 1.45431E-05
SCF Iteration   3: Energy = -113.8745546226130330   dE = -2.01030E-07   dRMS = 6.74540E-06
SCF Iteration   4: Energy = -113.8745546510455426   dE = -2.84325E-08   dRMS = 4.69966E-06
Performed QED-RHF on the following molecule


0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1

Total time for SCF iterations: 0.045 seconds 

QED-RHF   energy: -113.87455465 hartree
Psi4  SCF energy: -113.87722218 hartree
-113.8772221762759

-113.8772221762759

Start SCF iterations:

Canonical RHF One-electron energy = -218.2835951352540746
CQED-RHF One-electron energy = -218.2780272733479308
Nuclear repulsion energy = 31.7803657982661569
Dipole energy = 0.0000000000000000
SCF Iteration   1: Energy = -113.8745521325829202   dE = -1.13875E+02   dRMS = 3.86627E-05
SCF Iteration   2: Energy = -113.8745544215829284   dE = -2.28900E-06   dRMS = 1.45431E-05
SCF Iteration   3: Energy = -113.8745546226130330   dE = -2.01030E-07   dRMS = 6.74540E-06
SCF Iteration   4: Energy = -113.8745546510455426   dE = -2.84325E-08   dRMS = 4.69966E-06
Performed QED-RHF on the following molecule


0 1
O 0.0000000000 0.0000000000 5.91268220e-01
C 0.0000000000 0.0000000000 -5.90400099e-01
H 0.0000000000 9.32184336e-01 -1.17703144e+00
H 0.0000000000 -9.32184336e-01 -1.17703144e+00
no_reorient
symmetry c1

Total time for SCF iterations: 0.048 seconds 

QED-RHF   energy: -113.87455465 hartree
Psi4  SCF energy: -113.87722218 hartree
-113.8772221762759

In [19]:
f.close()

In [4]:
ndocc = wfn.nalpha()
nmo = wfn.nmo()
nvirt = nmo-ndocc

print(ndocc)
print(nvirt)
MO = cqed_dict['cqed_rhf_transformation_vectors'] 
MO_occ = MO[:,:ndocc]
MO_virt = MO[:, ndocc:]

CVEC = cqed_dict['cqed_cis eigenvectors']

nto_dict = get_nto(CVEC, MO_occ, MO_virt)
a = np.sum(nto_dict['weights LP0'])
b = np.sum(nto_dict['weights LP1'])
c = np.sum(nto_dict['weights UP0'])
d = np.sum(nto_dict['weights UP1'])
print(a+b+c+d)

print(nto_dict['weights LP0'])
print(nto_dict['weights LP1'])
print(nto_dict['weights UP0'])
print(nto_dict['weights UP1'])

8
4
1.0000723019948037
[9.99414791e-01 1.32494939e-06 5.77184657e-32 4.96132406e-34]
[5.83873678e-04 1.07430273e-08 4.80395718e-33 2.71081583e-35]
[5.97404250e-05 9.16978488e-06 3.17219099e-06 2.18157791e-07]
[1.26462254e-09 8.72866364e-11 7.62703332e-11 7.97794906e-12]


In [5]:
# get dictionary form of wavefunction
wfn_dict = psi4.core.Wavefunction.to_file(wfn)
#### uncomment Get orbitals from CQED
wfn_dict['matrix']['Ca'] = nto_dict['NTO UP1']
wfn_dict['matrix']['Cb'] = nto_dict['NTO UP1']
# update wfn object
wfn_cqed_cis = psi4.core.Wavefunction.from_file(wfn_dict) 


In [7]:
psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS'],
    'CUBEPROP_ORBITALS': [7,8,9,10],
})
cube = psi4.cubeprop(wfn_cqed_cis)

fortecubeview.plot()

CubeViewer: listing cube files from the directory .
Reading 4 cube files


HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Cube files:', options=('MO    7a (7-A)', 'MO    8a (8-A)', 'MO    9a…

In [ ]:
psi4.set_options({
    'CUBEPROP_TASKS': ['ORBITALS'],
    'CUBEPROP_ORBITALS': [5,6,7,8],
})
cube = psi4.cubeprop(wfn_cqed_cis)
mol.save_xyz_file("mgh.xyz", 1)
f = open("mgh.xyz")
xyz = f.read()
f.close()


In [ ]:
fnto = open("Psi_a_6_6-A.cube")
datanto = fnto.read()
fnto.close()
v = py3Dmol.view()
v.addVolumetricData(datanto, "cube", {'isoval': -0.075, 'color': "red", 'opacity': 0.80})
v.addVolumetricData(datanto, "cube", {'isoval': 0.075, 'color': "blue", 'opacity': 0.80})
v.addModel(xyz, 'mol')
v.setStyle({'stick':{}})
v.zoomTo()
v.show()